In [18]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
dataset = pd.read_csv('datasets/intermediate/chapter5_result.csv')

corr_matrix = dataset.corr()
corr_matrix

acc_z     acc_y     acc_x     gra_z     gra_y  \
acc_z                      1.000000  0.039339  0.048716 -0.105737 -0.061757   
acc_y                      0.039339  1.000000  0.098800  0.005956  0.057895   
acc_x                      0.048716  0.098800  1.000000 -0.073457 -0.094933   
gra_z                     -0.105737  0.005956 -0.073457  1.000000  0.289683   
gra_y                     -0.061757  0.057895 -0.094933  0.289683  1.000000   
...                             ...       ...       ...       ...       ...   
ori_qw_freq_4.7_Hz_ws_100  0.061424 -0.027869 -0.025099 -0.029730  0.016834   
ori_qw_freq_4.8_Hz_ws_100  0.093387  0.006736 -0.010054 -0.032193  0.014342   
ori_qw_freq_4.9_Hz_ws_100  0.067128  0.004056  0.012355 -0.039740  0.032757   
ori_qw_freq_5.0_Hz_ws_100  0.056269 -0.009802  0.006119 -0.039060  0.022626   
cluster                    0.013778  0.017547 -0.058788 -0.038224  0.216730   

                              gra_x     gyr_z     gyr_y     gyr_x     mag_z  \
acc_z                     -0.119518  0.123535 -0.041238 -0.151749  0.029690   
acc_y                      0.001831 -0.017919  0.073663  0.205471 -0.038039   
acc_x                     -0.118729  0.288578  0.233225  0.008162 -0.000260   
gra_z                      0.674290 -0.050840  0.006524  0.004179 -0.430558   
gra_y                      0.299209  0.001112  0.016357  0.015754 -0.004645   
...                             ...       ...       ...       ...       ...   
ori_qw_freq_4.7_Hz_ws_100 -0.023056  0.023989 -0.046879 -0.087179 -0.011622   
ori_qw_freq_4.8_Hz_ws_100 -0.019104 -0.012593 -0.036188 -0.045442 -0.011088   
ori_qw_freq_4.9_Hz_ws_100  0.009281 -0.023055 -0.045898 -0.050727  0.009308   
ori_qw_freq_5.0_Hz_ws_100  0.023846 -0.015715 -0.039059 -0.035693 -0.002912   
cluster                    0.000960  0.108988 -0.116929 -0.078991  0.108592   

                              mag_y     mag_x  mic_dBFS    ori_qz    ori_qy  \
acc_z                     -0.102340  0.022155 -0.058713  0.022161  0.014541   
acc_y                      0.016703 -0.021672  0.054073 -0.019254  0.016704   
acc_x                     -0.051463 -0.013795 -0.064706  0.012298  0.013062   
gra_z                     -0.061670 -0.080919 -0.121072  0.319610 -0.213557   
gra_y                      0.697991 -0.431181  0.181076  0.299935 -0.212124   
...                             ...       ...       ...       ...       ...   
ori_qw_freq_4.7_Hz_ws_100 -0.024345  0.049829  0.019175  0.067160 -0.015064   
ori_qw_freq_4.8_Hz_ws_100 -0.035922  0.059179  0.025944  0.081728 -0.025600   
ori_qw_freq_4.9_Hz_ws_100  0.005203  0.043826  0.005345  0.076911 -0.052208   
ori_qw_freq_5.0_Hz_ws_100 -0.016500  0.059014  0.004532  0.065190 -0.023568   
cluster                    0.230487  0.018027  0.228072 -0.063293  0.073737   

                             ori_qx    ori_qw  label_cycling  \
acc_z                      0.047060 -0.003882       0.105153   
acc_y                      0.027016  0.000911      -0.052349   
acc_x                     -0.067493  0.099458       0.079795   
gra_z                      0.349716  0.012388      -0.550708   
gra_y                      0.073744 -0.585224      -0.232888   
...                             ...       ...            ...   
ori_qw_freq_4.7_Hz_ws_100  0.153566 -0.270900      -0.001195   
ori_qw_freq_4.8_Hz_ws_100  0.150426 -0.280618      -0.003029   
ori_qw_freq_4.9_Hz_ws_100  0.158741 -0.301406      -0.004841   
ori_qw_freq_5.0_Hz_ws_100  0.154505 -0.284235      -0.004761   
cluster                    0.014815 -0.272309      -0.021651   

                           label_downstairs  label_onsubway  \
acc_z                             -0.012299        0.017836   
acc_y                             -0.018305       -0.001459   
acc_x                              0.035897        0.016484   
gra_z                              0.060603        0.647085   
gra_y                              0.188194       -0.161167   
...                                 

In [5]:
from util.common import GPU
import time
import pandas as pd
from Chapter7.PrepareDatasetForLearning import PrepareDatasetForLearning
from Chapter7.FeatureSelection import FeatureSelectionClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

prepare = PrepareDatasetForLearning()

N_FORWARD_SELECTION = 50
dataset = pd.read_csv('datasets/intermediate/chapter5_result.csv')

cols = dataset.columns[dataset.columns.str.contains('label')]
cols = ['label_cycling', 'label_downstairs', 'label_onsubway',
       'label_playing_phone', 'label_running', 'label_standing',
       'label_upstairs', 'label_walking']

class_labels = []
dataset[cols]
for index, row in dataset[cols].iterrows():
    class_label = row.idxmax()
    class_labels.append(class_label)

dataset['class'] = class_labels

dataset = dataset.drop('time', axis=1)
dataset = dataset.dropna()

for col in dataset.columns[dataset.columns.str.contains('label')]:
    dataset = dataset.drop(col, axis=1)

X = dataset.drop('class', axis=1)
y = dataset['class']
fsc = FeatureSelectionClassification()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Filter method
selected_features_filter = fsc.filter_method(20, X_train, y_train)
print("Selected features by filter method: ", selected_features_filter)

# Wrapper method
selected_features_wrapper = fsc.wrapper_method(20, X_train, y_train)
print("Selected features by wrapper method: ", selected_features_wrapper)

# Embedded method
selected_features_embedded = fsc.embedded_method(20, X_train, y_train)
print("Selected features by embedded method: ", selected_features_embedded)

Selected features by filter method:  ['acc_z_temp_mean_ws_300', 'acc_y_temp_mean_ws_300', 'acc_x_temp_mean_ws_300', 'gra_y_temp_mean_ws_300', 'gra_x_temp_mean_ws_300', 'mag_z_temp_mean_ws_300', 'mag_y_temp_mean_ws_300', 'mic_dBFS_temp_mean_ws_300', 'pca_3_temp_mean_ws_300', 'pca_4_temp_mean_ws_300', 'acc_z_temp_std_ws_300', 'acc_y_temp_std_ws_300', 'acc_x_temp_std_ws_300', 'gyr_z_temp_std_ws_300', 'gyr_y_temp_std_ws_300', 'mag_z_temp_std_ws_300', 'mag_y_temp_std_ws_300', 'mag_x_temp_std_ws_300', 'acc_z_freq_0.0_Hz_ws_100', 'gra_y_freq_0.0_Hz_ws_100']
Selected features by wrapper method:  ['acc_y_temp_mean_ws_300', 'gra_y_temp_mean_ws_300', 'gra_x_temp_mean_ws_300', 'gyr_z_temp_mean_ws_300', 'gyr_y_temp_mean_ws_300', 'gyr_x_temp_mean_ws_300', 'mag_z_temp_mean_ws_300', 'mag_y_temp_mean_ws_300', 'mag_x_temp_mean_ws_300', 'mic_dBFS_temp_mean_ws_300', 'ori_qz_temp_mean_ws_300', 'acc_y_freq_0.0_Hz_ws_100', 'ori_qx_max_freq', 'ori_qx_freq_weighted', 'ori_qx_pse', 'ori_qx_freq_0.0_Hz_ws_100', 

/Users/young/miniforge3/envs/ml4qs/lib/python3.8/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Selected features by embedded method:  ['pca_1', 'gra_x_temp_mean_ws_300', 'mag_y_temp_mean_ws_300', 'gyr_y_temp_std_ws_300', 'acc_y_freq_0.0_Hz_ws_100', 'pca_2', 'gra_z', 'gyr_y_freq_0.0_Hz_ws_100', 'gra_x_freq_0.0_Hz_ws_100', 'mag_z_temp_mean_ws_300', 'gyr_y_temp_mean_ws_300', 'acc_x', 'ori_qz', 'pca_1_temp_std_ws_300', 'acc_y_temp_std_ws_300', 'acc_y_temp_mean_ws_300', 'acc_z_temp_mean_ws_300', 'acc_y_freq_0.4_Hz_ws_100', 'gra_y_temp_mean_ws_300', 'mag_x_temp_mean_ws_300']


In [ ]:
# Selected features by filter method:  ['mag_z_temp_mean_ws_300', 'mag_y_temp_mean_ws_300', 'acc_z_temp_std_ws_300', 'acc_y_temp_std_ws_300', 'acc_x_temp_std_ws_300', 'gyr_z_temp_std_ws_300', 'gyr_y_temp_std_ws_300', 'mag_z_temp_std_ws_300', 'mag_y_temp_std_ws_300', 'acc_z_freq_0.0_Hz_ws_100']

# Selected features by wrapper method:  ['acc_y_temp_mean_ws_300', 'gyr_x_temp_mean_ws_300', 'mag_z_temp_mean_ws_300', 'mag_y_temp_mean_ws_300', 'ori_qz_temp_mean_ws_300', 'acc_y_freq_0.0_Hz_ws_100', 'ori_qy_freq_5.0_Hz_ws_100', 'ori_qx_max_freq', 'ori_qx_freq_weighted', 'ori_qx_pse']

In [2]:
selected_cols = ['mag_z_temp_mean_ws_300', 'mag_y_temp_mean_ws_300', 'acc_z_temp_std_ws_300', 'acc_y_temp_std_ws_300', 'acc_x_temp_std_ws_300', 'gyr_z_temp_std_ws_300', 'gyr_y_temp_std_ws_300', 'mag_z_temp_std_ws_300', 'mag_y_temp_std_ws_300', 'acc_z_freq_0.0_Hz_ws_100']